In [1]:
import requests
import bs4
from bs4 import BeautifulSoup
import pandas as pd
import re
from geopy.geocoders import Nominatim
from geopy.distance import great_circle
import time
import timeit
from IPython.core.display import Image
import numpy as np
from matplotlib import pyplot as plt
import cPickle

% matplotlib inline

#import textacy
#import Levenshtein

In [2]:
URL = 'http://www.viamichelin.com'
paths = pd.read_csv('/Users/Rebecca/DSI-projects/projects/capstone/assets/michelin_paths.csv')
paths = paths['0']
paths.reset_index(drop = True)
paths.columns = ['path']

test_paths = paths[0:10]

for i in test_paths:
    print i

/web/Restaurant/Penmarch-29760-Sterenn-12822-41102
/web/Restaurant/Saint_Jean_Cap_Ferrat-06230-Le_Cap-12838-41102
/web/Restaurant/Saint_Jean_Cap_Ferrat-06230-La_Voile_d_Or-12839-41102
/web/Restaurant/St_Jean_du_Bruel-12230-Midi_Papillon-12855-41102
/web/Restaurant/Saint_Jean_Pied_de_Port-64220-Les_Pyrenees-12858-41102
/web/Restaurant/Saint_Laurent_de_Mure-69720-Christian_Lavault-12878-41102
/web/Restaurant/Saint_Leonard_de_Noblat-87400-Le_Relais_St_Jacques-12885-41102
/web/Restaurant/St_Malo-35400-La_Grassinais-12895-41102
/web/Restaurant/Tilques-62500-Chateau_Tilques-12915-41102
/web/Restaurant/Saint_Pons-07580-Hostellerie_Gourmande_Mere_Biquette-12938-41102


In [3]:
lil_soup = []
for i in test_paths:
    time.sleep(1)
    requestlet = requests.get(URL + i)
    if str(requestlet) != '<Response [200]>':
        print URL + i
    souplet = BeautifulSoup(requestlet.content, "lxml")
    lil_soup.append(souplet)

In [20]:
def restaurantify(list_soup):
    
    '''This function creates a dataframe with restaurant observations and columns containing relevant information.'''
    
    name = []
    address = []
    cuisine = []
    blurb = []
    price = []
    author = []
    distinction = []
    standing = []
    photo = []
    add_info = []


    for tiny_soup in list_soup:
        
        if tiny_soup.findAll('div', class_ = 'datasheet-item datasheet-name') is None:
            name.append('empty')
            continue
        else:
            for rest_name in tiny_soup.findAll('div', class_ = 'datasheet-item datasheet-name'):
                name.append(rest_name.text)
        
        if tiny_soup.findAll('div', class_ = 'datasheet-item') is None:
            address.append('empty')
            continue
        else:    
            addresses = []
            for rest_address in tiny_soup.findAll('div', class_ = 'datasheet-item'):
                addresses.append(rest_address.text)
            address.append(addresses[1])
        
        if tiny_soup.findAll('div', class_ = 'datasheet-cooking-type') is None:
            cuisine.append('empty')
            continue
        else:
            for rest_cuisine in tiny_soup.findAll('div', class_ = 'datasheet-cooking-type'):
                cuisine.append(rest_cuisine.text)
        
        if tiny_soup.findAll('div', class_ = 'datasheet-description') is None:
            blurb.append('empty')
            continue
        else:
            for rest_blurb in tiny_soup.findAll('div', class_ = 'datasheet-description'):
                for j in rest_blurb.find_all('blockquote'):
                    blurb.append(j.text)
        
        if tiny_soup.findAll('div', class_ = 'datasheet-price') is None:
            price.append('empty')
            continue
        else:
            for rest_price in tiny_soup.findAll('div', class_ = 'datasheet-price'):
                price.append(rest_price.text)
    
        if tiny_soup.findAll('p', class_ = 'michelin-guide-inspectors') is None:
            author.append('empty')
            continue
        else:
            for rest_author in tiny_soup.findAll('p', class_ = 'michelin-guide-inspectors'):
                author.append(rest_author.text)
        
        if tiny_soup.findAll('div', class_ = 'poi-item-stars') is None:
            distinction.append('empty')
            continue
        else:
            distinctions = []
            for rest_distinction in tiny_soup.findAll('div', class_ = 'poi-item-stars'):
                for j in rest_distinction.find_all('span'):
                    distinctions.append(j['class'])
            distinction.append(distinctions)
        
        if tiny_soup.findAll('span', class_=re.compile('standing')) is None:
            standing.append('empty')
            continue
        else:
            for rest_standing_raw in tiny_soup.findAll('span', class_=re.compile('standing')):
                rest_standing = rest_standing_raw['class']
                standing.append(rest_standing)
        
        if tiny_soup.findAll('div', class_ = 'datasheet-more-info datasheet-photo clearfx') is None:
            photo.append('empty')
            continue
        else:
            for rest_photo in tiny_soup.findAll('div', class_ = 'datasheet-more-info datasheet-photo clearfx'):
                photo.append(rest_photo.text[0:13])
        if tiny_soup.findAll('div', class_ = 'datasheet-more-info clearfx') is None:
            info.append('empty')
            continue
        else:
            infos = []
            for rest_add_info in tiny_soup.findAll('div', class_ = 'datasheet-more-info clearfx'):
                infos.append(rest_add_info.text)
            add_info.append(infos)
        
    df = pd.DataFrame(zip(name, address, cuisine, blurb, price, author, distinction, standing, photo, add_info),\
                        columns = ['name', 'address', 'cuisine', 'blurb', 'price', 'author', 'distinction', \
                                   'standing', 'photo', 'add_info']) 
    print len(df)
    print 'name: %s' %len(name)
    print 'address: %s' %len(address)
    print 'cuisine: %s' %len(cuisine)
    print 'blurb: %s' %len(blurb)
    print 'price: %s' %len(price)
    print 'author: %s' %len(author)
    print 'distinction: %s' %len(distinction)
    print 'standing: %s' %len(standing)
    print 'photo: %s' %len(photo)
    print 'add_info: %s' %len(add_info)
    return df

In [9]:
def get_name(list_soup):
    name = []
    for tiny_soup in list_soup:
        names = []
        for rest_name in tiny_soup.findAll('div', class_ = 'datasheet-item datasheet-name'):
            names.append(rest_name.text)
        name.append(names) 
    return name

def get_address(list_soup):
    address = []
    for tiny_soup in list_soup:
        addresses = []
        for rest_address in tiny_soup.findAll('div', class_ = 'datasheet-item'):
            addresses.append(rest_na.text)
        